## THE ADVISORY FILE
#mock data

In [ ]:
import pandas as pd
import numpy as np

crops = ['Rice', 'Maize', 'Coffee', 'Cotton', 'Grapes']
data_size = 500

mock_data = {
    #generate random 500 integers b2n 0 and 139 
    'N': np.random.randint(0, 140, data_size),
    'P': np.random.randint(5, 145, data_size),       
    'K': np.random.randint(5, 205, data_size),
       #generate random 500 decimal  b2n 0 and 139 
    'temperature': np.random.uniform(15, 40, data_size),
    'humidity': np.random.uniform(15, 100, data_size),
    'ph': np.random.uniform(4.5, 9.0, data_size),
    'rainfall': np.random.uniform(20, 300, data_size),
    'label': np.random.choice(crops, data_size)
}
df= pd.DataFrame(mock_data)
df.to_csv('mock_crop_data.csv', index= False)
print(df.shape)
print('mock dataset created successfully!')

### advisory logic

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import joblib

df= pd.read_csv('mock_crop_data.csv')
x= df.drop('label', axis=1)
y=df['label']

model =RandomForestClassifier(n_estimators=100) #random forest with 100 decision trees
model.fit(x, y)

def get_farmer_advice(lat, lon, soil_type, n, p, k, ph):
    # For now, I will use mock weather values
    mock_temp = 25.5
    mock_humidity = 70.0
    mock_rainfall = 100.0
    #prediction
    prediction = model.predict([[n, p, k, mock_temp, mock_humidity, ph, mock_rainfall]])
    advice = {
        "location": f"Coordinates ({lat}, {lon})",
        "soil_analysis": f"Type: {soil_type}, PH: {ph}",
        "recommended_crop": prediction[0],
        "message":f"Based on your {soil_type} soil and current climate, we recommend planting {prediction[0]}."
    }
    return advice
print(get_farmer_advice(-1.28, 36.82, "loamy", 90, 40, 40, 6.5))
    